In [ ]:
%pip install openai==0.28
%pip install pytesseract pillow gradio


In [2]:
import os
import openai
import gradio as gr
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file
load_dotenv(find_dotenv())

# Set the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Function to extract text from image
def extract_text_from_image(image):
    try:
        # Preprocess the image
        img = image.convert('L')  # Convert to grayscale
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(2)  # Increase contrast
        img = img.filter(ImageFilter.SHARPEN)  # Sharpen the image

        # Perform OCR on the image
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        return f"Error during OCR: {e}"

# Function to use GPT-4 to identify marked fields in the extracted text
def identify_marked_fields_with_gpt(extracted_text):
    messages = [
        {"role": "system", "content": "You are a kind helpful assistant."},
        {"role": "user", "content": f"Here is the extracted text from an image: \n\n{extracted_text}\n\nIdentify and list all marked fields accurately. and for every marked field you find that corresponds to a diagnosis give the CIE-10 code"},
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4o",  # Use the appropriate model name
        messages=messages
    )

    reply = response.choices[0].message['content']
    return reply

# Main function to process the image and extract marked fields
def process_image(image):
    extracted_text = extract_text_from_image(image)
    if "Error" in extracted_text:
        return extracted_text
    marked_fields = identify_marked_fields_with_gpt(extracted_text)
    if "Error" in marked_fields:
        return marked_fields
    return marked_fields

# Create the Gradio interface
interface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Textbox(),
    title="Marked Fields Extractor",
    description="Upload an image to extract and identify marked fields using OCR and GPT-4."
)

# Launch the interface with public link
interface.launch(share=True)


Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://7d7047394cb44636fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [3]:
import os
import openai
import gradio as gr
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
import fitz  # PyMuPDF
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file
load_dotenv(find_dotenv())

# Set the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Function to convert PDF to images using PyMuPDF
def convert_pdf_to_images(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        images = []
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap()
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            images.append(img)
        return images
    except Exception as e:
        return f"Error during PDF conversion: {e}"

# Function to extract text from image
def extract_text_from_image(image):
    try:
        # Preprocess the image
        img = image.convert('L')  # Convert to grayscale
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(2)  # Increase contrast
        img = img.filter(ImageFilter.SHARPEN)  # Sharpen the image

        # Perform OCR on the image
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        return f"Error during OCR: {e}"

# Function to use GPT-4 to identify marked fields in the extracted text
def identify_marked_fields_with_gpt(extracted_text):
    try:
        messages = [
        {"role": "system", "content": "You are a kind helpful assistant."},
        {"role": "user", "content": f"Here is the extracted textand from an image: \n\n{extracted_text}\n\nIdentify and list all marked fields accurately.  for every marked field you find that corresponds to a diagnosis give the CIE-10 code"},
        ]

        response = openai.ChatCompletion.create(
            model="gpt-4o",  # Use the appropriate model name
            messages=messages
        )

        reply = response.choices[0].message['content']
        return reply
    except Exception as e:
        return f"Error during GPT-4 processing: {e}"

# Main function to process the PDF and extract marked fields
def process_pdf(pdf_file_path):
    # Convert PDF to images
    images = convert_pdf_to_images(pdf_file_path)
    if isinstance(images, str) and images.startswith("Error"):
        return images

    all_marked_fields = []

    for image in images:
        extracted_text = extract_text_from_image(image)
        if "Error" in extracted_text:
            return extracted_text
        marked_fields = identify_marked_fields_with_gpt(extracted_text)
        if "Error" in marked_fields:
            return marked_fields
        all_marked_fields.append(marked_fields)

    return "\n\n".join(all_marked_fields)

# Function to display images
def show_images(pdf_file_path):
    images = convert_pdf_to_images(pdf_file_path)
    if isinstance(images, str) and images.startswith("Error"):
        return []
    return images

# Create the Gradio interface
def interface_fn(pdf_file_path):
    text_output, images = process_pdf(pdf_file_path)
    return text_output, gr.update(visible=False), gr.update(visible=True)

def show_images_fn(pdf_file_path):
    images = show_images(pdf_file_path)
    return gr.update(visible=True, value=images)


interface = gr.Interface(
    fn=interface_fn,
    inputs=gr.File(type="filepath", label="Upload PDF"),
    outputs=[
        gr.Textbox(label="Extracted Marked Fields"),
        gr.Button("Show Images"),
        gr.Gallery(label="PDF Pages", visible=False)
    ],
    title="PDF Marked Fields Extractor",
    description="Upload a PDF to extract and identify marked fields using OCR and GPT-4. Click 'Show Images' to display the PDF pages."
)

interface.launch(share=True)



Running on local URL:  http://127.0.0.1:7865
Running on public URL: https://29b3b61a76531f6d64.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [4]:
import os
import openai
import gradio as gr
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
import fitz  # PyMuPDF
from dotenv import load_dotenv, find_dotenv
import json


In [5]:

load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

with open('./app/model/configs/gpt_config.json', 'r') as config_file:
    config = json.load(config_file)


def convert_pdf_to_images(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        images = []
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap()
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            images.append(img)
        return images
    except Exception as e:
        return f"Error during PDF conversion: {e}"

# Function to extract text from image
def extract_text_from_image(image):
    try:
        # Preprocess the image
        img = image.convert('L')  # Convert to grayscale
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(2)  # Increase contrast
        img = img.filter(ImageFilter.SHARPEN)  # Sharpen the image

        # Perform OCR on the image
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        return f"Error during OCR: {e}"

# Function to use GPT-4 to identify marked fields in the extracted text
def identify_marked_fields_with_gpt(extracted_text):
    try:
        messages = [
        {"role": "system", "content": "You are a kind helpful assistant."},
        {"role": "user", "content": f"Here is the extracted textand from an image: \n\n{extracted_text}\n\nIdentify and list all marked fields accurately.  for every marked field you find that corresponds to a diagnosis give the CIE-10 code"},
        ]

        response = openai.ChatCompletion.create(
            model=config.get("model"),
            messages=messages,
            temperature=config.get("temperature"),
            max_tokens=config.get("max_tokens"),
            top_p=config.get("top_p"),
            frequency_penalty=config.get("frequency_penalty"),
            presence_penalty=config.get("presence_penalty")
        )

        reply = response.choices[0].message['content']
        return reply
    except Exception as e:
        return f"Error during GPT-4 processing: {e}"


def process_pdf(pdf_file_path):


    images = convert_pdf_to_images(pdf_file_path)
    if isinstance(images, str) and images.startswith("Error"):
        return images, []

    all_marked_fields = []

    for image in images:
        extracted_text = extract_text_from_image(image)
        if "Error" in extracted_text:
            return extracted_text, []
        marked_fields = identify_marked_fields_with_gpt(extracted_text)
        if "Error" in marked_fields:
            return marked_fields, []
        all_marked_fields.append(marked_fields)

    return "\n\n".join(all_marked_fields), images

# Function to display images
def show_images(pdf_file_path):
    images = convert_pdf_to_images(pdf_file_path)
    if isinstance(images, str) and images.startswith("Error"):
        return []
    return images

# Create the Gradio interface
def interface_fn(pdf_file_path):
    text_output, images = process_pdf(pdf_file_path)
    return text_output, gr.update(visible=False), gr.update(visible=True)

def show_images_fn(pdf_file_path):
    images = show_images(pdf_file_path)
    return gr.update(visible=True, value=images)



In [6]:
with gr.Blocks() as interface:
    pdf_input = gr.File(type="filepath", label="Upload PDF")
    extract_button = gr.Button("Extract Marked Fields")
    text_output = gr.Textbox(label="Extracted Marked Fields")
    show_images_button = gr.Button("Show Images")
    image_gallery = gr.Gallery(label="PDF Pages", visible=False)

    extract_button.click(fn=interface_fn, inputs=pdf_input, outputs=[text_output, extract_button, show_images_button], api_name="extract")
    show_images_button.click(fn=show_images_fn, inputs=pdf_input, outputs=image_gallery)

interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://69ccd51db10d950c8b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [7]:
with gr.Blocks() as interface:
    gr.Markdown(
        """
        # PDF Marked Fields Extractor
        Upload a PDF to extract and identify marked fields using OCR and GPT-4o. 
        Click 'Submit' to extract the fields and 'Show Images' to display the PDF pages.
        """
    )

    pdf_input = gr.File(type="filepath", label="Upload PDF")
    with gr.Row():
        submit_button = gr.Button("Submit")
        clear_button = gr.Button("Clear")
    text_output = gr.Textbox(label="Extracted Marked Fields", placeholder="Marked fields will appear here...")
    show_images_button = gr.Button("Show Images", visible=True)
    image_gallery = gr.Gallery(label="PDF Pages", visible=True)

    submit_button.click(
        fn=interface_fn, 
        inputs=pdf_input, 
        outputs=[text_output, show_images_button],
        api_name="extract",
        
    )
    show_images_button.click(fn=show_images_fn, inputs=pdf_input, outputs=image_gallery)
    clear_button.click(fn=lambda: (None, gr.update(visible=True), gr.update(visible=True)), outputs=[text_output, show_images_button, image_gallery])

interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7867
Running on public URL: https://c05fa40a15aaf14f33.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [8]:

with gr.Blocks(css=".gradio-container {max-width: 800px; margin: auto;}") as interface:
    gr.Markdown(
        """
        # PDF Marked Fields Extractor
        Upload a PDF to extract and identify marked fields using OCR and GPT-4. 
        Click 'Submit' to extract the fields and 'Show Images' to display the PDF pages.
        """
    )

    pdf_input = gr.File(type="filepath", label="Upload PDF")
    with gr.Row():
        submit_button = gr.Button("Submit")
        clear_button = gr.Button("Clear")
    text_output = gr.Textbox(label="Extracted Marked Fields", placeholder="Marked fields will appear here...")
    show_images_button = gr.Button("Show Images", visible=False)
    image_gallery = gr.Gallery(label="PDF Pages", visible=False)

    submit_button.click(
        fn=interface_fn, 
        inputs=pdf_input, 
        outputs=[text_output, show_images_button]
    )
    show_images_button.click(fn=show_images_fn, inputs=pdf_input, outputs=image_gallery)
    clear_button.click(fn=lambda: (None, gr.update(visible=True), gr.update(visible=True)), outputs=[text_output, show_images_button, image_gallery])

interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7868
Running on public URL: https://6e7f7ea360cb291fd4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [9]:
css_path = "./styles/retro_style.css"

with gr.Blocks(css=open(css_path).read()) as interface:
    gr.Markdown(
        """
        # PDF Marked Fields Extractor
        Upload a PDF to extract and identify marked fields using OCR and GPT-4. 
        Click 'Submit' to extract the fields and 'Show Images' to display the PDF pages.
        """
    )

    pdf_input = gr.File(type="filepath", label="Upload PDF", elem_classes=["gr-file"])
    with gr.Row():
        submit_button = gr.Button("Submit", elem_classes=["gr-button"])
        clear_button = gr.Button("Clear", elem_classes=["gr-button"])
    text_output = gr.Textbox(label="Extracted Marked Fields", placeholder="Marked fields will appear here...", elem_classes=["gr-textbox"])
    show_images_button = gr.Button("Show Images", visible=False, elem_classes=["gr-button"])
    image_gallery = gr.Gallery(label="PDF Pages", visible=False, elem_classes=["gr-gallery"])

    submit_button.click(
        fn=interface_fn, 
        inputs=pdf_input, 
        outputs=[text_output, show_images_button]
    )
    show_images_button.click(fn=show_images_fn, inputs=pdf_input, outputs=image_gallery)
    clear_button.click(fn=lambda: (None, gr.update(visible=False), gr.update(visible=False)), outputs=[text_output, show_images_button, image_gallery])

interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7869
Running on public URL: https://919dfc18523ef20928.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
css_path = "./styles/futuristic_style.css"

with gr.Blocks(css=open(css_path).read()) as interface:
    gr.Markdown(
        """
        # PDF Marked Fields Extractor
        Upload a PDF to extract and identify marked fields using OCR and GPT-4. 
        Click 'Submit' to extract the fields and 'Show Images' to display the PDF pages.
        """
    )

    pdf_input = gr.File(type="filepath", label="Upload PDF", elem_classes=["gr-file"])
    with gr.Row():
        submit_button = gr.Button("Submit", elem_classes=["gr-button"])
        clear_button = gr.Button("Clear", elem_classes=["gr-button"])
    text_output = gr.Textbox(label="Extracted Marked Fields", placeholder="Marked fields will appear here...", elem_classes=["gr-textbox"])
    show_images_button = gr.Button("Show Images", visible=False, elem_classes=["gr-button"])
    image_gallery = gr.Gallery(label="PDF Pages", visible=False, elem_classes=["gr-gallery"])

    submit_button.click(
        fn=interface_fn, 
        inputs=pdf_input, 
        outputs=[text_output, show_images_button]
    )
    show_images_button.click(fn=show_images_fn, inputs=pdf_input, outputs=image_gallery)
    clear_button.click(fn=lambda: (None, gr.update(visible=False), gr.update(visible=False)), outputs=[text_output, show_images_button, image_gallery])

interface.launch(share=True)